# 本模块实现特征衍生

In [25]:
import pandas as pd
import numpy as np
# base
import math

import pandas as pd
import numpy as np
import sys
import datetime
import re
# data process
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import PolynomialFeatures, LabelBinarizer
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_string_dtype
import seaborn as sns
# model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegressionCV, LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import (RandomTreesEmbedding, RandomForestClassifier,
                              GradientBoostingClassifier)
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib
# model test
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

## 训练样本和预测样本概况

In [26]:
# 训练样本 ：(1368146, 55)，总计15000个不同的用户，日期为2015-3-1至2016-2-23，不均匀分布,正负bi：14230:770
# data_path = "./data/train.tsv"
# data_id = "./data/train_id.tsv"
data_path = "./data/train.tsv"
data_id = "./data/train_id.tsv"
dat = pd.read_csv(data_path, sep='\t')
dat_id = pd.read_csv(data_id,sep="\t")

data = pd.merge(dat,dat_id, how="inner", on="PERSONID")


print(data.shape)
data.head()

(1368146, 56)


,PERSONID,APPLYNO,FTR0,FTR1,FTR2,FTR3,FTR4,FTR5,FTR6,FTR7,...,FTR44,FTR45,FTR46,FTR47,FTR48,FTR49,FTR50,FTR51,CREATETIME,LABEL
0,8ba94305557a4ec72435a9eb78001350,20fab21284e57a4ccd3127553920e37b,0.050000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.050000,0.0,0.0,0.000000,0.050012,0.0,0.001532,A2B186C398E0D0,2015-09-14,0
1,8ba94305557a4ec72435a9eb78001350,73f32b8b6e199129abb8fe0cb6d6c867,0.033333,0.0,0.0,0.0,0.0,0.007829,0.0,0.0,...,0.033333,0.0,0.0,0.000000,0.033333,0.0,0.005156,A24B176C1239E0D0,2015-09-14,0
2,8ba94305557a4ec72435a9eb78001350,324da7a0b1a5c1d3b321685d734a9dc8,0.050000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.050000,0.0,0.0,0.000023,0.050000,0.0,0.000000,A0B0C6E0D0,2015-09-14,0
3,8ba94305557a4ec72435a9eb78001350,341ceb4ae7b0ca1c46653f5a116d4bd2,0.050000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.050000,0.0,0.0,0.000023,0.050000,0.0,0.000000,A0B0C6E0D0,2015-10-16,0
4,8ba94305557a4ec72435a9eb78001350,ca260c60e62de319e38a475e2455afb8,0.050000,0.0,0.0,0.0,0.0,0.007829,0.0,0.0,...,0.050000,0.0,0.0,0.000000,0.050000,0.0,0.005156,A24B176C1239E0D0,2015-10-16,0


In [27]:
# 需要预测的样本 ： (232502, 55)，总计2500个用户,日期为2015-3-1至2016-2-23，不均匀分布
data_path = "./data/test_A.tsv"
# data_path = "./data/test_A.tsv"
test_data = pd.read_csv(data_path,sep="\t")
# print test_data.head()
print(test_data.shape)
print(len(test_data.PERSONID.unique()))
# test_time = test_data.CREATETIME.value_counts().to_dict()
# sorted(test_time.keys())

(232502, 55)
2500


## 特征处理

In [28]:
cols = data.columns.tolist()
print(cols)
# print dat.dtypes
# print dat[u'PERSONID'].value_counts()
data.head()
# 字符型：u'PERSONID', u'APPLYNO'，u'FTR51', u'CREATETIME'
# 其他均为数值型


['PERSONID', 'APPLYNO', 'FTR0', 'FTR1', 'FTR2', 'FTR3', 'FTR4', 'FTR5', 'FTR6', 'FTR7', 'FTR8', 'FTR9', 'FTR10', 'FTR11', 'FTR12', 'FTR13', 'FTR14', 'FTR15', 'FTR16', 'FTR17', 'FTR18', 'FTR19', 'FTR20', 'FTR21', 'FTR22', 'FTR23', 'FTR24', 'FTR25', 'FTR26', 'FTR27', 'FTR28', 'FTR29', 'FTR30', 'FTR31', 'FTR32', 'FTR33', 'FTR34', 'FTR35', 'FTR36', 'FTR37', 'FTR38', 'FTR39', 'FTR40', 'FTR41', 'FTR42', 'FTR43', 'FTR44', 'FTR45', 'FTR46', 'FTR47', 'FTR48', 'FTR49', 'FTR50', 'FTR51', 'CREATETIME', 'LABEL']


,PERSONID,APPLYNO,FTR0,FTR1,FTR2,FTR3,FTR4,FTR5,FTR6,FTR7,...,FTR44,FTR45,FTR46,FTR47,FTR48,FTR49,FTR50,FTR51,CREATETIME,LABEL
0,8ba94305557a4ec72435a9eb78001350,20fab21284e57a4ccd3127553920e37b,0.050000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.050000,0.0,0.0,0.000000,0.050012,0.0,0.001532,A2B186C398E0D0,2015-09-14,0
1,8ba94305557a4ec72435a9eb78001350,73f32b8b6e199129abb8fe0cb6d6c867,0.033333,0.0,0.0,0.0,0.0,0.007829,0.0,0.0,...,0.033333,0.0,0.0,0.000000,0.033333,0.0,0.005156,A24B176C1239E0D0,2015-09-14,0
2,8ba94305557a4ec72435a9eb78001350,324da7a0b1a5c1d3b321685d734a9dc8,0.050000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.050000,0.0,0.0,0.000023,0.050000,0.0,0.000000,A0B0C6E0D0,2015-09-14,0
3,8ba94305557a4ec72435a9eb78001350,341ceb4ae7b0ca1c46653f5a116d4bd2,0.050000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.050000,0.0,0.0,0.000023,0.050000,0.0,0.000000,A0B0C6E0D0,2015-10-16,0
4,8ba94305557a4ec72435a9eb78001350,ca260c60e62de319e38a475e2455afb8,0.050000,0.0,0.0,0.0,0.0,0.007829,0.0,0.0,...,0.050000,0.0,0.0,0.000000,0.050000,0.0,0.005156,A24B176C1239E0D0,2015-10-16,0


In [29]:
# 分开数值特征和字符特征
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_string_dtype
string_fea = [] 
numeric_fea = []
que_fea = []
for c in cols:
    if is_string_dtype(data[c]):
        string_fea.append(c)
    elif is_numeric_dtype(data[c]):
        numeric_fea.append(c)
    else:
        que_fea.append(c)
print("字符特征/数值特征/待定特征数量依次为： ",list(map(len,[string_fea,numeric_fea,que_fea])))

字符特征/数值特征/待定特征数量依次为：  [4, 52, 0]


In [30]:
string_fea

['PERSONID', 'APPLYNO', 'FTR51', 'CREATETIME']

### 数值型特征处理

In [41]:
print(numeric_fea)
# 查看各字段的方差
data[numeric_fea].describe()

['FTR0', 'FTR1', 'FTR2', 'FTR3', 'FTR4', 'FTR5', 'FTR6', 'FTR7', 'FTR8', 'FTR9', 'FTR10', 'FTR11', 'FTR12', 'FTR13', 'FTR14', 'FTR15', 'FTR16', 'FTR17', 'FTR18', 'FTR19', 'FTR20', 'FTR21', 'FTR22', 'FTR23', 'FTR24', 'FTR25', 'FTR26', 'FTR27', 'FTR28', 'FTR29', 'FTR30', 'FTR31', 'FTR32', 'FTR33', 'FTR34', 'FTR35', 'FTR36', 'FTR37', 'FTR38', 'FTR39', 'FTR40', 'FTR41', 'FTR42', 'FTR43', 'FTR44', 'FTR45', 'FTR46', 'FTR47', 'FTR48', 'FTR49', 'FTR50', 'LABEL']


,FTR0,FTR1,FTR2,FTR3,FTR4,FTR5,FTR6,FTR7,FTR8,FTR9,...,FTR42,FTR43,FTR44,FTR45,FTR46,FTR47,FTR48,FTR49,FTR50,LABEL
count,1.368146e+06,1.368146e+06,1.368146e+06,1.368146e+06,1.368146e+06,1.368146e+06,1.368146e+06,1.368146e+06,1.368146e+06,1.368146e+06,...,1.368146e+06,1.368146e+06,1.368146e+06,1.368146e+06,1.368146e+06,1.368146e+06,1.368146e+06,1.368146e+06,1.368146e+06,1.368146e+06
mean,5.855092e-02,6.901618e-06,5.358761e-05,6.820737e-06,1.254527e-04,2.288102e-04,7.983492e-06,1.250401e-04,5.184892e-05,3.727709e-04,...,4.984105e-02,3.698680e-04,5.347194e-02,7.962530e-06,2.468331e-05,9.940218e-05,5.705913e-02,1.005303e-05,1.201223e-04,9.531073e-02
std,7.568552e-02,6.942325e-04,7.072420e-04,6.939824e-04,7.400491e-04,6.973571e-04,7.484879e-04,7.400275e-04,8.324062e-04,6.429483e-04,...,2.935995e-02,6.443573e-04,6.112476e-02,2.416337e-04,7.127198e-04,7.143739e-04,7.458931e-02,7.144584e-04,7.319925e-04,2.936438e-01
min,4.347826e-03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,4.347826e-03,0.000000e+00,4.347826e-03,0.000000e+00,0.000000e+00,0.000000e+00,4.347826e-03,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.000000e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,5.000000e-02,0.000000e+00,5.000000e-02,0.000000e+00,0.000000e+00,0.000000e+00,5.000000e-02,0.000000e+00,0.000000e+00,0.000000e+00
50%,5.000000e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,5.000000e-02,0.000000e+00,5.000000e-02,0.000000e+00,0.000000e+00,5.762762e-06,5.000000e-02,0.000000e+00,0.000000e+00,0.000000e+00
75%,5.000000e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.249506e-04,...,5.000000e-02,6.189709e-04,5.000000e-02,0.000000e+00,0.000000e+00,2.305105e-05,5.000000e-02,0.000000e+00,0.000000e+00,0.000000e+00
max,9.099994e-01,2.536921e-01,1.525279e-01,2.542501e-01,1.640532e-01,3.617471e-02,1.294412e-01,1.641625e-01,4.514852e-01,7.379243e-02,...,8.999999e-01,7.200450e-02,9.526309e-01,6.236983e-02,1.325515e-01,1.171762e-01,6.999997e-01,1.319481e-01,1.325492e-01,1.000000e+00


In [ ]:
# 没有空缺值
# for c in numeric_fea:
#     print c,data[data[numeric_fea[0]].isnull()].shape

# 天花板地板处理，做对数处理（是否正态分布）
def mm_replace(x):
    if x > maxx:
#         print x
        x = maxx
    elif x < minn:
#         print x
        x = minn
    else:
        x = x
    return x

for c in numeric_fea:
    # 天花板地板处理
    minn = data[c].quantile(0.0001)
    maxx = data[c].quantile(0.9999)
    # print "processing col: ",c
    data[c] = map(mm_replace,data[c])

    # log处理
    data[c] = map(lambda x:np.log(x + 1),data[c])
    

### 数值特征组合

In [64]:
def deal_ftr51(ser):
      return  [len(np.unique(ser)),len(np.unique(ser))]
    

func_list = [np.mean, np.sum, np.std]

# func_dict = {i : func_list for i in numeric_fea if i != 'LABEL'}
# func_dict['APPLYNO'] = ['count']
func_dict['FTR51'] = [deal_ftr51]
df_agg = data.groupby("PERSONID").agg(func_dict)

#data[numeric_fea+["PERSONID"]].groupby("PERSONID").agg(f)
#df_list = [0]*len(func_list)
#n = 0
#for f in func_list: 
#     df_list[n] = data[numeric_fea+["PERSONID"]].groupby("PERSONID").agg(f)
#     df_list[n].columns = [ col_name + "_" + f.__name__.split("_")[1] for col_name in df_list[n].columns] 
#     n += 1

In [65]:
df_51 = data[['PERSONID','APPLYNO','FTR51']]
df_51

,PERSONID,APPLYNO,FTR51
0,8ba94305557a4ec72435a9eb78001350,20fab21284e57a4ccd3127553920e37b,A2B186C398E0D0
1,8ba94305557a4ec72435a9eb78001350,73f32b8b6e199129abb8fe0cb6d6c867,A24B176C1239E0D0
2,8ba94305557a4ec72435a9eb78001350,324da7a0b1a5c1d3b321685d734a9dc8,A0B0C6E0D0
3,8ba94305557a4ec72435a9eb78001350,341ceb4ae7b0ca1c46653f5a116d4bd2,A0B0C6E0D0
4,8ba94305557a4ec72435a9eb78001350,ca260c60e62de319e38a475e2455afb8,A24B176C1239E0D0
5,8ba94305557a4ec72435a9eb78001350,55eac334641dd47c43d5f95c5e2b6e6f,A2B186C398E0D0
6,8ba94305557a4ec72435a9eb78001350,ce6cf113cde7df53b776dafb77049487,A0B0C6E0D0
7,8ba94305557a4ec72435a9eb78001350,f699af5bb8589446c7a1d06632b35f7b,A24B3634C1239E0D0
8,8ba94305557a4ec72435a9eb78001350,b55e5b0bd3817ffb897e1dcaf430f3ca,A2B287C398E0D0
9,8ba94305557a4ec72435a9eb78001350,54d4bf98bd7baadd5306f41ab1e3af89,A24B3634C1239E0D0


In [66]:
df_agg.shape
df_agg.head()

FTR0                     FTR1            \
                                      mean       sum       std mean  sum  std   
PERSONID                                                                        
000e3ee6e04a8b18871e1c85ffab6e49  0.052956  4.289436  0.032572  0.0  0.0  0.0   
000eba50dc8c2e3899070c6b82986814  0.052685  5.321175  0.045622  0.0  0.0  0.0   
00139506cbd54b670e1e6dec234190e1  0.048932  1.908333  0.004752  0.0  0.0  0.0   
001de94e8d0d152e26ad87cb409003f9  0.059370  3.443451  0.085962  0.0  0.0  0.0   
001df6145dea5c48e752d8471544969f  0.055077  3.800345  0.042554  0.0  0.0  0.0   

                                      FTR2                     FTR3  \
                                      mean       sum       std mean   
PERSONID                                                              
000e3ee6e04a8b18871e1c85ffab6e49  0.000000  0.000000  0.000000  0.0   
000eba50dc8c2e3899070c6b82986814  0.000000  0.000000  0.000000  0.0   
00139506cbd54b670e1e6dec234190e1  0.000000  0.000000  0.000000  0.0   
001de94e8d0d152e26ad87cb409003f9  0.000000  0.000000  0.000000  0.0   
001df6145dea5c48e752d8471544969f  0.000046  0.003171  0.000166  0.0   

                                    ...         FTR48           FTR49       \
                                    ...           sum       std  mean  sum   
PERSONID                            ...                                      
000e3ee6e04a8b18871e1c85ffab6e49    ...      3.839515  0.009380   0.0  0.0   
000eba50dc8c2e3899070c6b82986814    ...      4.833848  0.008579   0.0  0.0   
00139506cbd54b670e1e6dec234190e1    ...      2.575009  0.104263   0.0  0.0   
001de94e8d0d152e26ad87cb409003f9    ...      2.776833  0.008175   0.0  0.0   
001df6145dea5c48e752d8471544969f    ...      3.235738  0.010234   0.0  0.0   

                                          FTR50                     APPLYNO  \
                                  std      mean       sum       std   count   
PERSONID                                                                      
000e3ee6e04a8b18871e1c85ffab6e49  0.0  0.000000  0.000000  0.000000      81   
000eba50dc8c2e3899070c6b82986814  0.0  0.000000  0.000000  0.000000     101   
00139506cbd54b670e1e6dec234190e1  0.0  0.000000  0.000000  0.000000      39   
001de94e8d0d152e26ad87cb409003f9  0.0  0.000138  0.008029  0.000306      58   
001df6145dea5c48e752d8471544969f  0.0  0.000177  0.012198  0.000370      69   

                                      FTR51  
                                 deal_ftr51  
PERSONID                                     
000e3ee6e04a8b18871e1c85ffab6e49   [42, 42]  
000eba50dc8c2e3899070c6b82986814   [50, 50]  
00139506cbd54b670e1e6dec234190e1   [21, 21]  
001de94e8d0d152e26ad87cb409003f9   [31, 31]  
001df6145dea5c48e752d8471544969f   [37, 37]  

[5 rows x 155 columns]

In [33]:
data.head()

,PERSONID,APPLYNO,FTR0,FTR1,FTR2,FTR3,FTR4,FTR5,FTR6,FTR7,...,FTR44,FTR45,FTR46,FTR47,FTR48,FTR49,FTR50,FTR51,CREATETIME,LABEL
0,8ba94305557a4ec72435a9eb78001350,20fab21284e57a4ccd3127553920e37b,0.050000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.050000,0.0,0.0,0.000000,0.050012,0.0,0.001532,A2B186C398E0D0,2015-09-14,0
1,8ba94305557a4ec72435a9eb78001350,73f32b8b6e199129abb8fe0cb6d6c867,0.033333,0.0,0.0,0.0,0.0,0.007829,0.0,0.0,...,0.033333,0.0,0.0,0.000000,0.033333,0.0,0.005156,A24B176C1239E0D0,2015-09-14,0
2,8ba94305557a4ec72435a9eb78001350,324da7a0b1a5c1d3b321685d734a9dc8,0.050000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.050000,0.0,0.0,0.000023,0.050000,0.0,0.000000,A0B0C6E0D0,2015-09-14,0
3,8ba94305557a4ec72435a9eb78001350,341ceb4ae7b0ca1c46653f5a116d4bd2,0.050000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.050000,0.0,0.0,0.000023,0.050000,0.0,0.000000,A0B0C6E0D0,2015-10-16,0
4,8ba94305557a4ec72435a9eb78001350,ca260c60e62de319e38a475e2455afb8,0.050000,0.0,0.0,0.0,0.0,0.007829,0.0,0.0,...,0.050000,0.0,0.0,0.000000,0.050000,0.0,0.005156,A24B176C1239E0D0,2015-10-16,0


In [11]:
df_list[0].head()

,FTR0_mean,FTR1_mean,FTR2_mean,FTR3_mean,FTR4_mean,FTR5_mean,FTR6_mean,FTR7_mean,FTR8_mean,FTR9_mean,...,FTR41_mean,FTR42_mean,FTR43_mean,FTR44_mean,FTR45_mean,FTR46_mean,FTR47_mean,FTR48_mean,FTR49_mean,FTR50_mean
PERSONID,,,,,,,,,,,,,,,,,,,,,
000e3ee6e04a8b18871e1c85ffab6e49,0.051176,0.0,0.000000,0.0,0.000000,0.000318,0.0,0.000000,0.000010,0.000531,...,0.000459,0.054573,0.000519,0.048995,0.0,0.0,0.000010,0.046272,0.0,0.000000
000eba50dc8c2e3899070c6b82986814,0.050606,0.0,0.000000,0.0,0.000000,0.000198,0.0,0.000000,0.000509,0.000426,...,0.000392,0.046715,0.000435,0.046715,0.0,0.0,0.000004,0.046716,0.0,0.000000
00139506cbd54b670e1e6dec234190e1,0.047762,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000734,...,0.000634,0.047762,0.000716,0.047762,0.0,0.0,0.000026,0.060527,0.0,0.000000
001de94e8d0d152e26ad87cb409003f9,0.055318,0.0,0.000000,0.0,0.000000,0.000322,0.0,0.000000,0.000007,0.000367,...,0.000421,0.046734,0.000359,0.046734,0.0,0.0,0.000028,0.046735,0.0,0.000138
001df6145dea5c48e752d8471544969f,0.052909,0.0,0.000046,0.0,0.000371,0.000327,0.0,0.000371,0.000019,0.000375,...,0.000354,0.047148,0.000371,0.045780,0.0,0.0,0.000036,0.045780,0.0,0.000177


In [19]:
# 将列表中DF连接成一个DATAFRAME
df_all = df_list[0]
print(df_all.shape)
for i in range(1,len(df_list)-1):
    data = pd.merge(dat,dat_id, how="inner", on="PERSONID")
    print df_list[i].shape
    df_all = pd.merge(df_all,df_list[i], how="inner", on="PERSONID")
print df_all.shape
# 添加 count列
df_count = pd.DataFrame(df_list[3]["FTR0_count"])
df_all = pd.merge(df_all,dd, how="inner", on="PERSONID")
df_all.rename(columns = {"FTR0_count":"count"},inplace=True)
print(df_all.shape)
print(df_all.columns.tolist())

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(df_list[i].shape)? (<ipython-input-19-1ffb31c3b0f3>, line 6)

In [48]:

# 保存数值特征结果
# df_all.head()
df_all = pd.merge(df_all,dat_id,how="inner",on="PERSONID")
# df_all.drop(["LABEL_x","LABEL_y"],axis=1,inplace=True)
df_all.to_csv("./data/numeric_fea.csv",index=None)
df_all.shape

(15000, 156)

In [51]:
data_fea = pd.read_csv("./data/numeric_fea.csv")
print data_fea.columns.tolist()

['PERSONID', 'FTR0_mean', 'FTR1_mean', 'FTR2_mean', 'FTR3_mean', 'FTR4_mean', 'FTR5_mean', 'FTR6_mean', 'FTR7_mean', 'FTR8_mean', 'FTR9_mean', 'FTR10_mean', 'FTR11_mean', 'FTR12_mean', 'FTR13_mean', 'FTR14_mean', 'FTR15_mean', 'FTR16_mean', 'FTR17_mean', 'FTR18_mean', 'FTR19_mean', 'FTR20_mean', 'FTR21_mean', 'FTR22_mean', 'FTR23_mean', 'FTR24_mean', 'FTR25_mean', 'FTR26_mean', 'FTR27_mean', 'FTR28_mean', 'FTR29_mean', 'FTR30_mean', 'FTR31_mean', 'FTR32_mean', 'FTR33_mean', 'FTR34_mean', 'FTR35_mean', 'FTR36_mean', 'FTR37_mean', 'FTR38_mean', 'FTR39_mean', 'FTR40_mean', 'FTR41_mean', 'FTR42_mean', 'FTR43_mean', 'FTR44_mean', 'FTR45_mean', 'FTR46_mean', 'FTR47_mean', 'FTR48_mean', 'FTR49_mean', 'FTR50_mean', 'FTR0_sum', 'FTR1_sum', 'FTR2_sum', 'FTR3_sum', 'FTR4_sum', 'FTR5_sum', 'FTR6_sum', 'FTR7_sum', 'FTR8_sum', 'FTR9_sum', 'FTR10_sum', 'FTR11_sum', 'FTR12_sum', 'FTR13_sum', 'FTR14_sum', 'FTR15_sum', 'FTR16_sum', 'FTR17_sum', 'FTR18_sum', 'FTR19_sum', 'FTR20_sum', 'FTR21_sum', 'FTR22_

In [52]:
data_fea.head()

,PERSONID,FTR0_mean,FTR1_mean,FTR2_mean,FTR3_mean,FTR4_mean,FTR5_mean,FTR6_mean,FTR7_mean,FTR8_mean,...,FTR43_std,FTR44_std,FTR45_std,FTR46_std,FTR47_std,FTR48_std,FTR49_std,FTR50_std,count,LABEL
0,000e3ee6e04a8b18871e1c85ffab6e49,0.051176,0.0,0.000000,0.0,0.000000,0.000318,0.0,0.000000,0.000010,...,0.000838,0.022296,0.0,0.0,0.000017,0.009094,0.0,0.000000,81.0,0
1,000eba50dc8c2e3899070c6b82986814,0.050606,0.0,0.000000,0.0,0.000000,0.000198,0.0,0.000000,0.000509,...,0.000536,0.008315,0.0,0.0,0.000005,0.008315,0.0,0.000000,101.0,0
2,00139506cbd54b670e1e6dec234190e1,0.047762,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000642,0.004575,0.0,0.0,0.000038,0.077352,0.0,0.000000,39.0,0
3,001de94e8d0d152e26ad87cb409003f9,0.055318,0.0,0.000000,0.0,0.000000,0.000322,0.0,0.000000,0.000007,...,0.000430,0.007916,0.0,0.0,0.000026,0.007916,0.0,0.000306,58.0,0
4,001df6145dea5c48e752d8471544969f,0.052909,0.0,0.000046,0.0,0.000371,0.000327,0.0,0.000371,0.000019,...,0.000456,0.009923,0.0,0.0,0.000158,0.009923,0.0,0.000370,69.0,0


In [57]:
data_fea.FTR11_mean.value_counts()

0.0    15000
Name: FTR11_mean, dtype: int64

In [53]:
import sys
sys.path.append("./data/work/")
from mean_encode import MeanEncode,ColDiscretizate
base_fea = ["PERSONID","LABEL"]
code_fea = []
for col_name in data_fea.columns.tolist():
    if col_name in base_fea:
        continue
    print col_name
    dis_mode = ColDiscretizate(data_fea, col_name, 'LABEL', './none')
    data_fea[col_name+"_code"] = dis_mode.transform()

    encode_model = MeanEncode(data_fea, encode_col = col_name, encode_label = 'LABEL', save_file = './none', k = 15, f = 5)
    data_fea[col_name+"_code"] = encode_model.transform()
    code_fea.append(col_name+"_code")

FTR0_mean


D:\soft\anaconda2\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


FTR1_mean


./data/work/mean_encode.py:42: RuntimeWarning: overflow encountered in exp
  return np.round(1.0/(1 + np.exp((n - self.k)/self.f)), 6)


FTR2_mean
FTR3_mean
FTR4_mean
FTR5_mean
FTR6_mean
FTR7_mean
FTR8_mean
FTR9_mean
FTR10_mean
FTR11_mean


IndexError: list index out of range